# 2. Azure 솔루션을 활용한 Advanced RAG 구현

## 2.1 Azure OpenAI On Your Data

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
search_endpoint = os.getenv("AZURE_SEARCH_ENDPOINT")
search_key = os.getenv("AZURE_SEARCH_API_KEY")
aoai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
aoai_key = os.getenv("AZURE_OPENAI_API_KEY")
embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME")
chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")

In [ ]:
# Import libraries
from openai import AzureOpenAI

# Set up the Azure OpenAI client
openai_client = AzureOpenAI(
     api_version="2024-05-01-preview",
     azure_endpoint=aoai_endpoint,
     api_key=aoai_key
 )

deployment_name = chat_deployment

response = openai_client.chat.completions.create(
    messages=[
        {   "role": "system",
            "content": "너는 인사 지원 에이전트야. 친절하게 이모티콘을 많이써서 답변하고, 참고해야 하는 자료 중에 관련 내용이 없다면 모르겠다고 답변할것. 그리고 관련 문의는 HR팀에게 해야 한다고 답변할 것."
        },
        {
            "role": "user",
            "content": "치과에서 임플란트랑 충치 치료를 최근에 했는데 사내 실비 보험으로 청구 가능한가요? 그리고 수술 관련 때문에 입원을 해야 할 것 같은데 병가를 사용할 수 있나요?"
        }
    ],
    model=deployment_name,
    temperature=0,
    extra_body={
        "data_sources": [
            {
                "type": "azure_search",
                "parameters": {
                    "endpoint": search_endpoint,
                    "index_name": "aoai-dev-day-index",
                    "authentication": {
                        "type": "api_key",
                        "key": search_key,
                        },
                    "query_type": "vector",
                    "embedding_dependency": {
                        "type": "deployment_name",
                        "deployment_name": embedding_deployment,
                        "dimensions": 1536
                    },
                    "top_n_documents": 4,
                    "include_contexts": ["citations", "intent", "all_retrieved_documents"]
                }
            }
        ]
    }
)

print(response.choices[0].message.content)

In [ ]:
response.choices[0].message.context['intent']

In [ ]:
response.choices[0].message.context['citations']

In [ ]:
response.choices[0].message.context['all_retrieved_documents']